In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import omegaconf
from datetime import datetime

from typing import Union, List
from copy import deepcopy
from tdc.utils import retrieve_benchmark_names

from graphium.config._loader import (
    load_datamodule,
    load_metrics,
    load_architecture,
    load_predictor,
    load_trainer,
    save_params_to_wandb,
    load_accelerator,
    load_yaml_config,
)

# Fine-tuning on the TDC ADMET benchmarking group

[TDC](https://tdcommons.ai/) hosts a variety of ML-ready datasets and benchmarks for ML for drug discovery. The [TDC ADMET benchmarking group](https://tdcommons.ai/benchmark/admet_group/overview/) is a popular collection of benchmarks for evaluating new _foundation models_ (see e.g. [MolE](https://arxiv.org/abs/2211.02657)) due to the variety and relevance of the included tasks.

The ADMET benchmarking group is integrated in `graphium` through the `ADMETBenchmarkDataModule` data-module. This notebook shows how to easily fine-tune and test a model using that data-module. 

<div style="background-color: #fff3cd; border-radius: 10px; border-color: #ffeeba; padding: 20px; margin: 20px 0;  color: #856404">
    <b>NOTE:</b> This notebook is still <i>work in progress</i>. While the <b>fine-tuning logic is unfinished</b>, the notebook does demo how one could use the data-module to easily loop over each of the datasets in the benchmarking group and get the prescribed train-test split. Once the fine-tuning logic is finalized, we will finish this notebook and officially provide it as a tutorial within Graphium. 
</div>

In [3]:
# First, let's read the yaml configuration file
with open("../expts/configs/config_tdc_admet_demo.yaml", "r") as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## Get all TDC benchmark names

In [4]:
benchmarks = retrieve_benchmark_names("admet_group")
len(benchmarks)

22

While there is a total of 22, let's just use two for practicality sake: One regression and one classification task! 

In [5]:
benchmarks = ["caco2_wang", "hia_hou"]

## Initialize all training components per task
**NOTE**: Since we do not have fine-tuning logic, this for now just creates a new model. Ultimately, we will want to use fine-tuning code to evaluate how well the pre-trained model transfers to downstream tasks. 

In [6]:
def training_testing_loop(cfg):
    """
    Simple loop to train a model from scratch and test it. 
    """
    
    # Initialize object from config
    cfg, accelerator_type = load_accelerator(cfg)
    datamodule = load_datamodule(cfg, accelerator_type)
    model_class, model_kwargs = load_architecture(cfg, in_dims=datamodule.in_dims)
    metrics = load_metrics(cfg)
    
    # Prepare data
    datamodule.prepare_data()
    
    # Initialize the predictor
    predictor = load_predictor(
        cfg,
        model_class,
        model_kwargs,
        metrics,
        datamodule.get_task_levels(),
        accelerator_type,
        datamodule.featurization,
        datamodule.task_norms
    )
    
    # Initialize the trainer
    date_time_suffix = datetime.now().strftime("%d.%m.%Y_%H.%M.%S")
    trainer = load_trainer(cfg, "tdc-admet", accelerator_type, date_time_suffix)
        
    # Train
    predictor.set_max_nodes_edges_per_graph(datamodule, stages=["train", "val"])
    trainer.fit(model=predictor, datamodule=datamodule)
    
    # Test
    predictor.set_max_nodes_edges_per_graph(datamodule, stages=["test"])
    results = trainer.test(model=predictor, datamodule=datamodule)
    
    return results


In [7]:
def filter_cfg_based_on_benchmark_name(config, names: Union[List[str], str]):
    """
    Filter a base config for the full TDC ADMET benchmarking group to only 
    have settings related to a subset of the endpoints
    """
    
    if config["datamodule"]["module_type"] != "ADMETBenchmarkDataModule":
        raise ValueError("You can only use this method for the `ADMETBenchmarkDataModule`")
        
    if isinstance(names, str):
        names = [names]
    
    def _filter(d):
        return {k: v for k, v in d.items() if k in names}
         
    cfg = deepcopy(config)
    
    # Update the datamodule arguments
    cfg["datamodule"]["args"]["tdc_benchmark_names"] = names
    
    # Filter the relevant config sections
    cfg["architecture"]["task_heads"] = _filter(cfg["architecture"]["task_heads"])
    cfg["predictor"]["metrics_on_progress_bar"] = _filter(cfg["predictor"]["metrics_on_progress_bar"])
    cfg["predictor"]["loss_fun"] = _filter(cfg["predictor"]["loss_fun"])
    cfg["metrics"] = _filter(cfg["metrics"])
    
    return cfg

In [8]:
results = {}

for name in benchmarks: 
    
    # Run the training-testing loop
    cfg = filter_cfg_based_on_benchmark_name(config, name)
    benchmark_results = training_testing_loop(cfg)
    
    # Extract the main metric from the config
    metric = cfg["predictor"]["metrics_on_progress_bar"][name][0]
    key = f"graph_{name}/{metric}/test"
    results[f"{name}/{metric}"] = benchmark_results[0][key]

2023-07-13 14:02:22.438 | INFO     | graphium.data.datamodule:__init__:2425 - Preparing the TDC ADMET Benchmark Group splits for each of the 1 benchmarks.
wandb: Currently logged in as: cwognum (valence-ood). Use `wandb login --relogin` to force relogin
wandb: WARNING Path logs/tdc-admet-demo/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path logs/tdc-admet-demo/wandb/ wasn't writable, using system temp directory


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-07-13 14:02:28.065 | INFO     | graphium.data.datamodule:setup:1168 - -------------------
MultitaskDataset
	about = training set
	num_graphs_total = 634
	num_nodes_total = 18351
	max_num_nodes_per_graph = 67
	min_num_nodes_per_graph = 2
	std_num_nodes_per_graph = 11.441048173903344
	mean_num_nodes_per_graph = 28.944794952681388
	num_edges_total = 39466
	max_num_edges_per_graph = 144
	min_num_edges_per_graph = 2
	std_num_edges_per_graph = 25.32877270037731
	mean_num_edges_per_graph = 62.24921135646688
-------------------

2023-07-13 14:02:28.065 | INFO     | graphium.data.datamodule:setup:1169 - -------------------
MultitaskDataset
	about = validation set
	num_graphs_total = 91
	num_nodes_total = 2791
	max_num_nodes_per_graph = 67
	min_num_nodes_per_graph = 13
	std_num_nodes_per_graph = 11.461406761225364
	mean_num_nodes_per_graph =

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
2023-07-13 14:02:49.408 | INFO     | graphium.data.datamodule:setup:1168 - -------------------
MultitaskDataset
	about = training set
	num_graphs_total = 634
	num_nodes_total = 18351
	max_num_nodes_per_graph = 67
	min_num_nodes_per_graph = 2
	std_num_nodes_per_graph = 11.441048173903344
	mean_num_nodes_per_graph = 28.944794952681388
	num_edges_total = 39466
	max_num_edges_per_graph = 144
	min_num_edges_per_graph = 2
	std_num_edges_per_graph = 25.32877270037731
	mean_num_edges_per_graph = 62.24921135646688
-------------------

2023-07-13 14:02:49.409 | INFO     | graphium.data.datamodule:setup:1169 - -------------------
MultitaskDataset
	about = validation set
	num_graphs_total = 91
	num_nodes_total = 2791
	max_num_nodes_per_graph = 67
	min_num_nodes_per_graph = 13
	std_num_nodes_per_graph = 11.461406761225364
	mean_num_nodes_per_graph = 30.67032967032967
	num_edges_total = 6034
	max_num_edges_per_graph = 148
	min_num_edges_per_graph = 28


...

Training: 0it [00:00, ?it/s]

2023-07-13 14:02:52.078 | WARNING  | graphium.trainer.predictor_summaries:get_metrics_logs:273 - Error for metric graph_hia_hou/mcc/train. NaN is returned. Exception: stack expects a non-empty TensorList
`Trainer.fit` stopped: `max_epochs=10` reached.
2023-07-13 14:03:06.289 | INFO     | graphium.data.datamodule:setup:1168 - -------------------
MultitaskDataset
	about = training set
	num_graphs_total = 403
	num_nodes_total = 9065
	max_num_nodes_per_graph = 101
	min_num_nodes_per_graph = 7
	std_num_nodes_per_graph = 8.379156239363269
	mean_num_nodes_per_graph = 22.49379652605459
	num_edges_total = 19420
	max_num_edges_per_graph = 220
	min_num_edges_per_graph = 14
	std_num_edges_per_graph = 18.678530787820403
	mean_num_edges_per_graph = 48.188585607940446
-------------------

2023-07-13 14:03:06.290 | INFO     | graphium.data.datamodule:setup:1169 - -------------------
MultitaskDataset
	about = validation set
	num_graphs_total = 58
	num_nodes_total = 1431
	max_num_nodes_per_graph = 67
	m

Testing: 0it [00:00, ?it/s]

2023-07-13 14:03:07.087 | WARNING  | graphium.trainer.predictor_summaries:get_metrics_logs:273 - Error for metric graph_hia_hou/mcc/test. NaN is returned. Exception: stack expects a non-empty TensorList


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃           Test metric            ┃           DataLoader 0           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         gpu_allocated_GB         │      0.0018811225891113281       │
│    graph_hia_hou/BCELoss/test    │        0.5804430842399597        │
│   graph_hia_hou/accuracy/test    │        0.7692307829856873        │
│     graph_hia_hou/auprc/test     │        0.8519759774208069        │
│     graph_hia_hou/auroc/test     │        0.7008230686187744        │
│     graph_hia_hou/loss/test      │        0.5804430842399597        │
│      graph_hia_hou/mcc/test      │               nan                │
│   graph_hia_hou/mean_pred/test   │        0.8799146413803101        │
│  graph_hia_hou/mean_target/test  │        0.7692307829856873        │
│  graph_hia_hou/median_pred/test  │        0.8844738006591797        │
│ graph_hia_hou/median_target/test │               1.0                │
│   graph_hia_hou/std_pred/test    │       0.010796700604259968       │
│  graph_hia_hou/std_target/test   │       0.42313718795776367        │
│            loss/test             │        0.5804430842399597        │
└──────────────────────────────────┴──────────────────────────────────┘

In [9]:
print(omegaconf.OmegaConf.to_yaml(results))

caco2_wang/mae: 2.312683582305908
hia_hou/auroc: 0.7008230686187744



The End. 